In [3]:
import re
import os

In [169]:
ccmatch = re.compile(r"(?:loan|credit agreement)\s*(?:numbers?|no[.]?)\s*(ibrd)?\s*([0-9\s]{4,5}(?:-?[0-9]?)*)[-\s]*([a-z]{2,4})")

grantmatch = re.compile(r"grant\s*number\s*(tf[0-9]*)[-\s]*([a-z]{2,4})")

In [100]:
source_dir = "world_bank_loans_txt"

In [160]:
id_date_regex = re.compile(r"([0-9]{4})\_(\w*)\_([0-9]{1,2})\_([0-9]*)")

In [196]:
file_list = os.listdir(source_dir)
bad_captures = []
country_codes = dict()
dates = dict()
for f in file_list:
    if f[-3:] != 'txt':
        continue
    info = id_date_regex.match(f)
    id = info.groups()[3]
    y = info.groups()[0]
    m = info.groups()[1]
    d = info.groups()[2]
    dates[id] = (y,m,d)
    with open(source_dir + '/' + f,'r') as fh:
        agreement = fh.read()
        captures = ccmatch.search(agreement.lower())
        if captures is None:
            captures = grantmatch.search(agreement.lower())
            if captures is None:
                bad_captures.append(f)
            else:
                country_codes[id] = captures.groups()[1]
        else:
            country_codes[id] = captures.groups()[2]

In [197]:
bad_captures

['2014_december_9_318201468006913878_official-documents-scf-loan-agreement-for-tf016083-closing-package.txt',
 '2018_november_27_243281543417298669_official-documents-amended-and-restated-scf-ppcr-loan-agreement-for-tf016912-closing-package.txt',
 '1995_march_23_450521468335049833_conformed-copy--c2676--basic-education-reform-project--loan-agreement.txt',
 '2019_august_29_768791568310323989_official-documents-loan-agreement-for-cfr-srep-loan-tf0a7640-closing-package.txt',
 '1994_june_21_740261468038036933_gabon--economic-recovery-loan-:-loan-3759--loan-agreement--conformed.txt',
 '1994_june_29_850181468048557222_lebanon--revenue-enhancement-and-fiscal-management-technical-assistance-project-:-loan-3770--loan-agreement--conformed.txt',
 '2017_may_22_415241496780518663_official-documents-scf-loan-agreement-for-tf019211.txt',
 '2015_july_2_504891468052795717_official-documents-loan-agreement-for-ctf-loan-tf019266-closing-package.txt',
 '2004_june_30_904411468267559962_loan-agremeent-l4740

In [164]:
country_codes.values()

dict_values(['yu', 'pol', 'cha', 'cn', 'yu', 'cn', 'cha', 'lt', 'al', 'ph', 'br', 'al', 'hr', 'cha', 'id', 'ind', 'pak', 'ba', 'br', 'co', 'br', 'br', 'png', 'ind', 'yf', 'br', 'ar', 'cha', 'kz', 'ind', 'ge', 'br', 'pk', 'ph', 'in', 'gu', 'yf', 'do', 'ar', 'vn', 'ge', 'cn', 'tun', 'pak', 'ao', 'ee', 'in', 'ind', 'ind', 'id', 'br', 'pe', 'in', 'sk', 'le', 'mk', 'cha', 'cha', 'al', 'ma', 'cha', 'mor', 'br', 'hr', 'in', 'ko', 'co', 'cn', 'pak', 'mor', 'id', 'cha', 'pol', 'ph', 'me', 'ec', 'ph', 'co', 'pan', 'in', 'cob', 'jo', 'yf', 'ba', 'br', 'gt', 'al', 'ua', 'al', 'ma', 'pe', 'al', 'cn', 'sk', 'in', 'az', 'br', 'ph', 'cha', 'ko', 'ind', 'in', 'ua', 'hr', 'cr', 'jo', 'co', 'cha', 'cn', 'cn', 'ar', 'pol', 'tun', 'es', 'cha', 'sa', 'mor', 'in', 'br', 'ph', 'lb', 'yf', 'ind', 'tr', 'ph', 'in', 'ua', 'in', 'br', 'mor', 'ph', 'mx', 'tun', 'me', 'ro', 'pl', 'uy', 'ph', 'slu', 'pa', 'co', 'ur', 'ph', 'jo', 'mu', 'mx', 'ph', 'ec', 'mx', 'cn', 'am', 'in', 'jo', 'ind', 'do', 'ind', 'pak', 'kz', '

In [194]:
f = source_dir + "/2010_july_21_376051489609974334_ctf-loan-agreement-for-tf096291-mx-conformed.txt"
with open(f) as fh:
    agreement = fh.read()
agreement = agreement.lower()

In [195]:
agreement[:1000]

' \n\n \n\nconformed copy \n\nctf loan number tf096291 \n\nclean technology fund loan agreement \n \n\n(urban transport transformation project) \n \n\ninternational bank for reconstruction \nand development  \nacting as an implementing entity of the clean technology fund \n \n\nbanco nacional de obras y servicios publicos, s.n.c. \n \n\nbetween \n \n\nand \n \n\ndated july 21, 2010 \n\n\x0cctf loan number tf096291 \n\n \n\n \n\n \n\nclean technology fund loan agreement \n\n \nagreement dated july 21, 2010, entered into between: \ninternational  bank  for  reconstruction  and  development \n(“world  bank”),  acting  not  in  its  individual  capacity  but  solely  in  its  capacity  as  an \nimplementing entity of the clean technology fund (“ctf”) and banco nacional \nde obras y servicios publicos, s.n.c. (“borrower”). \n \n(a)  whereas by an agreement of even date herewith between the united mexican \nstates  (the  guarantor),  through  shcp,  and  the  bank  (the  guarantee  agreement